In [13]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import *

vectorizer= TfidfVectorizer()
news_train = fetch_20newsgroups(subset='train')
W_train = vectorizer.fit_transform(news_train.data).T  

model=NMF(n_components=10, init='random', random_state=0)
Xtemp=model.fit_transform(W_train) 
H=model.components_

news_test = fetch_20newsgroups(subset='test') 
W_test = vectorizer.transform(news_test.data).T  

In [15]:
import numpy as np
import numpy.linalg as la
import numpy.random as rn
import matplotlib.pyplot as plt

y = W_test[1:1000,500]
X = Xtemp[1:1000, :]
print(y.shape)
print(X.shape)

(999, 1)
(999, 10)


In [16]:
def proj(v, z=1):
    n = v.shape[0]
    # check if v is already in the simplex
    if (np.sum(v) == z) and (v >= 0).all():
        return v
    
    u = np.sort(v)[::-1]
    
    usum = 0
    rho = 0
    for j in range(n):
        usum += u[j]
        cond = u[j] - 1/(j+1)*(usum - z) > 0
        
        if cond:
            rho = j
    
    theta = 1/(rho+1)*(np.sum(u[0:rho+1]) - z)
    
    w = (v - theta).clip(min=0)

In [23]:
def projsubgrad(beta, X, y, t):
    eta = 1e-3/np.sqrt(t+1)

    v = beta - eta*np.dot(X.T, np.sign(np.dot(X, beta)-y))

    beta = proj(v)
    
    return beta

In [18]:
def mirror(beta, X, y, t):
    eta = 1e-3/np.sqrt(t+1)
    
    beta_phi = np.zeros(beta.shape[0])
    for i in range(beta.shape[0]):
        if beta[i] == 0:
            beta_phi[i] = 1
        else:
            beta_phi[i] = np.log(beta[i]) + 1

    v_phi = beta_phi - eta*np.dot(X.T, np.sign(np.dot(X, beta)-y))
    v = np.exp(v_phi - 1)
    
    beta = v/np.sum(np.abs(v))
    return beta

In [19]:
def descent(update, X, y, T=int(1e2)):
    beta = np.zeros(X.shape[1])
    obj = []
    for i in range(beta.shape[0]):
        beta[i] = i
    
    beta = beta/np.sum(beta)
    
    for t in range(T):
        beta = update(beta, X, y, t)
        
        if(t % 1 == 0) or (t == T - 1):
            obj.append(la.norm(np.dot(X, beta) - y, 1))
            
            assert not np.isnan(obj[-1])
            
    return beta, obj

In [ ]:
beta_sg, obj_sg = descent(projsubgrad, X, y)

In [ ]:
beta_md, obj_md = descent(mirror, X, y)

When using all of the data, my computer freezes and after running for 15 minutes it required a hard reset on my machine. I'm fairly confident the code as the alogrithms are fairly straight forward, but I cannot, unfortunately, show results.